In [66]:
import requests
import openai
from dotenv import load_dotenv

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [67]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [69]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [70]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [71]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [6]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for rank, hit in enumerate(response['hits']['hits']):
        print(f"Score for result {rank+1} is: {hit['_score']}")
        result_docs.append(hit['_source'])
    
    return result_docs

In [7]:
results = elastic_search("How do I execute a command in a running docker container?")

Score for result 1 is: 84.050095
Score for result 2 is: 51.04628
Score for result 3 is: 49.938507
Score for result 4 is: 45.275463
Score for result 5 is: 45.255775


In [72]:
def elastic_search_with_filter(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for rank, hit in enumerate(response['hits']['hits']):
        print(f"Score for result {rank+1} is: {hit['_score']}")
        result_docs.append(hit['_source'])
    
    return result_docs

In [73]:
search_results = elastic_search_with_filter("How do I execute a command in a running docker container?")

Score for result 1 is: 84.050095
Score for result 2 is: 51.04628
Score for result 3 is: 49.938507


In [74]:
search_results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [75]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()

    context_docs = []
    for doc in search_results:
        formated_context = context_template.format(question=doc['question'], text=doc['text'])
        context_docs.append(formated_context)

    context = "\n\n".join(context_docs)

    prompt = prompt_template.format(question=query, context=context).strip()
    print(f"Number of characters: {len(prompt)}")
    return prompt

In [76]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [77]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

def rag(query):
    search_results = elastic_search_with_filter(query)
    prompt = build_prompt(query, search_results)
    tokens = encoding.encode(prompt)
    num_tokens = len(tokens)
    print("Number of tokens:", num_tokens)
    response = llm(prompt)
    return response


In [78]:
query = "How do I execute a command in a running docker container?"

answer = rag(query)

Score for result 1 is: 84.050095
Score for result 2 is: 51.04628
Score for result 3 is: 49.938507
Number of characters: 1462
Number of tokens: 322
